In [3]:
import os
import pandas as pd
import zipfile
from io import StringIO

In [9]:
def extract_and_process_data(root_folder):
    all_data = []
    
    # Traverse through the nested folder structure
    for year in os.listdir(root_folder):
        year_path = os.path.join(root_folder, year)
        if not os.path.isdir(year_path):
            continue
        
        for month in os.listdir(year_path):
            month_path = os.path.join(year_path, month)
            if not os.path.isdir(month_path):
                continue
            
            first_zip = True
            for file in os.listdir(month_path):
                if file.endswith(".zip"):
                    zip_path = os.path.join(month_path, file)
                    if first_zip:
                        print(f"Processing Year/Month: {month}")
                        first_zip = False
                    
                    with zipfile.ZipFile(zip_path, 'r') as z:
                        for filename in z.namelist():
                            with z.open(filename) as f:
                                df = pd.read_csv(f, delimiter=',', dtype={"PQ_description": str})  # Change "column_name" to actual column name
                                # Filter the required data
                                df_filtered = df[
                                    (df["PQ_unit"] == "cm") & 
                                    (df["PQ_name"] == "淹水深度") & 
                                    (df["value"] > 0)
                                ][["station_id", "timestamp", "value"]]
                                
                                all_data.append(df_filtered)
    
    # Combine all data into a single DataFrame
    final_df = pd.concat(all_data, ignore_index=True)
    return final_df

root_folder = './Data/raw_data'
records = extract_and_process_data(root_folder)

Processing Year/Month: 202207
Processing Year/Month: 202209
Processing Year/Month: 202208
Processing Year/Month: 202201
Processing Year/Month: 202206
Processing Year/Month: 202212
Processing Year/Month: 202203
Processing Year/Month: 202204
Processing Year/Month: 202205
Processing Year/Month: 202202
Processing Year/Month: 202211
Processing Year/Month: 202210
Processing Year/Month: 201907
Processing Year/Month: 201909
Processing Year/Month: 201908
Processing Year/Month: 201906
Processing Year/Month: 201901
Processing Year/Month: 201912
Processing Year/Month: 201904
Processing Year/Month: 201903
Processing Year/Month: 201902
Processing Year/Month: 201905
Processing Year/Month: 201911
Processing Year/Month: 201910
Processing Year/Month: 202112
Processing Year/Month: 202107
Processing Year/Month: 202109
Processing Year/Month: 202108
Processing Year/Month: 202101
Processing Year/Month: 202106
Processing Year/Month: 202111
Processing Year/Month: 202110
Processing Year/Month: 202103
Processing

/var/folders/t2/sk5q5lf54ds4j5jwz4wynx740000gn/T/ipykernel_13683/938937747.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_data, ignore_index=True)


In [11]:
# Check for duplicates in records
num_duplicates = records.duplicated().sum()
print(f"Number of duplicate rows in records: {num_duplicates}")

# Drop duplicates in records
records = records.drop_duplicates()

Number of duplicate rows in records: 737728


In [13]:
records.head()

,station_id,timestamp,value
0,38505796-1525-4c8b-9d5c-27fea47db00f,2022-07-21 00:00:31.039,0.019717
1,38505796-1525-4c8b-9d5c-27fea47db00f,2022-07-21 00:09:31.974,0.020480
2,38505796-1525-4c8b-9d5c-27fea47db00f,2022-07-21 00:11:01.615,0.020215
3,38505796-1525-4c8b-9d5c-27fea47db00f,2022-07-21 00:19:31.676,0.019876
4,38505796-1525-4c8b-9d5c-27fea47db00f,2022-07-21 00:21:01.238,0.018207


In [77]:
sensors = pd.read_csv('./Data/sensors.csv')
sensors = sensors.drop(columns=['ciOrgname', 'OrgName', 'Code', 'PQ_id', 'FullName', 'Description', 'ciCategory', 'CategoryInfos_Name', 
                                'Address', 'PQ_name', 'station_name'])
sensors.drop(sensors[sensors['SIUnit  '] != 'cm'].index, inplace=True)
sensors = sensors[~((sensors["Longitude"] == 120) & (sensors["Latitude"] == 23))]

In [78]:
sensors

,station_id,Longitude,Latitude,SIUnit
0,648c0721-9ae3-4a3b-9007-31dd06a5f293,120.241250,23.450130,cm
1,b320d298-d3aa-4954-874a-79696f550efa,120.188995,23.428696,cm
2,c7c0c173-be6c-4fd2-b743-921d987e7330,120.392550,23.483112,cm
3,bc5af470-def9-4712-95da-8cc29c35fd60,120.160995,23.508854,cm
4,54c2b021-edc6-418f-bff5-ec96067b24e6,120.433920,23.441912,cm
...,...,...,...,...
2301,7709e9f3-d69e-4906-a12d-31d897de59fe,120.350815,23.979065,cm
2302,8975cb1d-3b8a-4868-96d2-4ce3da4c7250,120.237380,23.415445,cm
2303,0728ae5a-0da2-4138-9ba1-92f9788d0210,120.302850,23.464588,cm
2304,e41ad5de-70e4-4857-87d3-873884a8fb77,120.333360,23.488043,cm


In [79]:
# Check for duplicates in sensors
num_duplicates = sensors.duplicated().sum()
print(f"Number of duplicate rows in sensors: {num_duplicates}")

# Drop duplicates in sensors
sensors = sensors.drop_duplicates()

Number of duplicate rows in sensors: 19


In [80]:
# Left-join dataframes on station_id
df = records.merge(sensors, on='station_id', how='left')

In [81]:
df

,station_id,timestamp,value,Longitude,Latitude,SIUnit
0,38505796-1525-4c8b-9d5c-27fea47db00f,2022-07-21 00:00:31.039,0.019717,NaN,NaN,NaN
1,38505796-1525-4c8b-9d5c-27fea47db00f,2022-07-21 00:09:31.974,0.020480,NaN,NaN,NaN
2,38505796-1525-4c8b-9d5c-27fea47db00f,2022-07-21 00:11:01.615,0.020215,NaN,NaN,NaN
3,38505796-1525-4c8b-9d5c-27fea47db00f,2022-07-21 00:19:31.676,0.019876,NaN,NaN,NaN
4,38505796-1525-4c8b-9d5c-27fea47db00f,2022-07-21 00:21:01.238,0.018207,NaN,NaN,NaN
...,...,...,...,...,...,...
9656118,a2015019-1ecc-48a2-938d-393a289e7a9a,2020-12-11 10:05:17.262,1.022019,120.35272,23.652205,cm
9656119,ae9eb0c9-a435-4b9a-b5e2-08d00c43b231,2020-12-11 07:30:00,1.500928,120.47167,23.642970,cm
9656120,ae9eb0c9-a435-4b9a-b5e2-08d00c43b231,2020-12-11 08:50:00,1.516509,120.47167,23.642970,cm
9656121,039025b9-0b8e-4683-b143-29cfaa9a86bc,2020-12-11 09:41:31,2.200000,NaN,NaN,NaN


In [82]:
# Drop NAs
print("Number of missing values per column before dropping:")
print(df.isna().sum())  # Counts NaN values for each column

df = df.dropna(subset=['Longitude', 'Latitude'])

Number of missing values per column before dropping:
station_id          0
timestamp           0
value               0
Longitude     2723756
Latitude      2723756
SIUnit        2723756
dtype: int64


In [83]:
# Number of unique stations
df['station_id'].value_counts()

station_id
a3550f5f-34eb-4364-bd5d-c51a115ea6b7    292299
1c8f0f5d-7940-4734-b59a-4f0031ef5304    275050
e375e985-80ae-4b6e-a30f-8c061351f2a7    261460
e040cbc5-e893-4a32-b3e1-e5b3232b00e6    258824
969cc500-c74c-4aea-9f2a-687fa2135d92    251539
                                         ...  
f7787713-a6e7-4ab2-85e4-4a7c06fea968         1
4f8e8092-fb61-4696-9f93-4f5fe4a4cccf         1
58da7833-59a2-4058-87d3-52acca22fac2         1
706d660c-b9c8-4966-8134-52ab6624ea67         1
852ab211-20ce-4d29-ab00-ddda2333438e         1
Name: count, Length: 1267, dtype: int64

In [84]:
# Distribution of flood depth
df['value'].describe()

count    6.932367e+06
mean     7.937204e+02
std      1.044377e+03
min      1.000000e-04
25%      5.000000e-01
50%      6.890357e+00
75%      1.910000e+03
max      6.038774e+05
Name: value, dtype: float64

In [85]:
df

,station_id,timestamp,value,Longitude,Latitude,SIUnit
559,d83ac636-3d28-43fe-96a9-5c33dde8aebe,2022-07-21 00:08:57.2,0.001000,120.69100,23.903200,cm
560,d83ac636-3d28-43fe-96a9-5c33dde8aebe,2022-07-21 00:18:57.382,0.001000,120.69100,23.903200,cm
561,d83ac636-3d28-43fe-96a9-5c33dde8aebe,2022-07-21 00:28:58.358,0.001000,120.69100,23.903200,cm
562,d83ac636-3d28-43fe-96a9-5c33dde8aebe,2022-07-21 00:38:58.793,0.001000,120.69100,23.903200,cm
563,d83ac636-3d28-43fe-96a9-5c33dde8aebe,2022-07-21 00:48:59.713,0.001000,120.69100,23.903200,cm
...,...,...,...,...,...,...
9656116,5abec613-f581-4bf9-b41c-dc0619c950c4,2020-12-11 11:30:00,1.545227,120.30970,23.654995,cm
9656117,5abec613-f581-4bf9-b41c-dc0619c950c4,2020-12-11 15:03:18.016,1.504207,120.30970,23.654995,cm
9656118,a2015019-1ecc-48a2-938d-393a289e7a9a,2020-12-11 10:05:17.262,1.022019,120.35272,23.652205,cm
9656119,ae9eb0c9-a435-4b9a-b5e2-08d00c43b231,2020-12-11 07:30:00,1.500928,120.47167,23.642970,cm


In [86]:
# Export to csv
df.to_csv("df.csv", index=False)